# Capital Gain Calculator (FIFO Method)
Adapted from VBA to Python

### Structures
1. Transaction queue
    - HashMap (key is Asset, dict is list of two deques)
2. Transactions
    - 6 properties: Timestamp, Asset (eg. BTC), Type (buy or sell), Units, Total Amount ($), IRS ID (eg. Gemi 1)

### Outline
1. Validate transaction log CSV
    - Sort CSV by Timestamp property
    - Ensure valid Type property and corresponding Units property sign
    - Ensure other properties properties
2. Input data from transaction log CSV file into a buy transaction queue and sell transaction queue for each asset (eg. transaction log with BTC and ETH transactions -> BTC-Buy, BTC-Sell, ETH-Buy, Eth-Sell)
3. For each asset, run FIFO algorithm
    - While there are still transactions in the sell transaction queue, match front of sell queue with front of buy queue transaction (verifying buy transaction Timestamp property is before sell transaction)
    - When a match is found, add write buy-sell transaction to output CSV file
    - Update remaining balance (Units and Total Amount property) on buy/sell transaction and/or remove empty transaction(s) from respective queue
    - Run until the sell transaction queue is empty, the remaining buy transaction(s) are the carryover
4. Create summary report


Question:

FIFO method only matters in the case that there is carryover. If there is no carryover, that means every buy has a sell?? Could we just say in the case of carryover, look back at where the most recent buy transactions were and conclude that those are the carryover inventory still in possession. 

Actually I'm not entirely sure, it may be that the timing of certain buy/sell affects capital gain.

# 1. Read and Validate Transaction Log CSV

In [368]:
import csv
from datetime import datetime

# Open CSV file
file_name = "large_example.csv"
file = open(file_name, 'r')
transaction_log = csv.DictReader(file)

# Sort transaction log by Timestamp property (must have format mm/dd/yyyy hh:mm:ss)
transaction_log = sorted(transaction_log, key=lambda d: datetime.strptime(d['Timestamp'], "%m/%d/%Y %H:%M:%S"))

# Change datatypes of Units and Total Amount to float
for tx in transaction_log:
    tx['Units'] = float(tx['Units'])
    tx['Total Amount'] = float(tx['Total Amount'])

# 2. Input Data into Transaction Queues

In [369]:
from collections import deque

# Create Transaction Queues
asset_map = {}

# Define buy and sell constants
BUY = 0
SELL = 1

for tx in transaction_log:
    # Create buy and sell deques for each asset
    asset = tx['Asset']
    if asset not in asset_map:
        buy_deque = deque()
        sell_deque = deque()
        asset_map[asset] = (buy_deque, sell_deque)
    
    # Add transaction to respective deque
    if tx['Type'] == "Buy":
        asset_map[asset][BUY].appendleft(tx)
    else:
        asset_map[asset][SELL].appendleft(tx)
        
# Close CSV File
file.close()

# 3. Run FIFO Transaction Matching

In [370]:

# Loop through each asset
for asset in asset_map:
    # Run until there are no more transactions in sell deque
    print(asset)
    print("--------------------------------------------")
    capital_gain, buy_volume, sell_volume = 0, 0, 0
    
    while asset_map[asset][SELL]:
        sell_tx = asset_map[asset][SELL].pop()
        
        # rounding down to 0????
        if abs(sell_tx['Units']) < .0000001:
            print(sell_tx['Units'])
            continue
        
        # to-do: catch error here, asset_map[asset][BUY] must have a tx with units greater than sell_tx
        
        buy_tx = asset_map[asset][BUY].pop()
        
        match = {}
        match['Buy IRS ID'] = buy_tx['IRS ID']
        match['Buy Timestamp'] = buy_tx['Timestamp']
        match['Sell Timestamp'] = sell_tx['Timestamp']
        
        
        # to-do: catch error here, buy_tx['Timestamp'] must be before sell_tx['Timestamp']
        assert (datetime.strptime(buy_tx['Timestamp'], "%m/%d/%Y %H:%M:%S") < datetime.strptime(sell_tx['Timestamp'], "%m/%d/%Y %H:%M:%S"))
        
        if abs(sell_tx['Units']) > buy_tx['Units']:
            pro_rata_sale_price = buy_tx['Units'] / abs(sell_tx['Units']) * sell_tx['Total Amount']
            
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = pro_rata_sale_price
            match['Basis'] = buy_tx['Total Amount']
            
            sell_tx['Units'] = sell_tx['Units'] + buy_tx['Units']
            sell_tx['Total Amount'] = sell_tx['Total Amount'] - pro_rata_sale_price
            
            asset_map[asset][SELL].append(sell_tx)
        
        elif abs(sell_tx['Units']) < buy_tx['Units']:
            pro_rata_basis = abs(sell_tx['Units']) / buy_tx['Units'] * buy_tx['Total Amount']
            
            match['Units'] = abs(sell_tx['Units'])
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = pro_rata_basis
            
            buy_tx['Units'] = buy_tx['Units'] + sell_tx['Units']
            buy_tx['Total Amount'] = buy_tx['Total Amount'] - pro_rata_basis

            asset_map[asset][BUY].append(buy_tx)
        
        # transaction units are the same
        else:
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = buy_tx['Total Amount']
            
        
        match['Capital Gain'] = match['Sale Price'] - match['Basis']
        # print(match['Sale Price'], "-", match['Basis'], "=", match['Capital Gain'])
        #print("-- Units:", match['Units'], "-- Capital Gain:", match['Sale Price'], "-", match['Basis'], "=", match['Capital Gain'], "Buy:", buy_tx['IRS ID'], "-- Sell:", sell_tx['IRS ID'])
        #print("--------------------------------------------")
        capital_gain += match['Capital Gain']
        buy_volume += match['Basis']
        sell_volume += match['Sale Price']
        

        
    # Carryover is remaining transactions in buy deque
    while asset_map[asset][BUY]:
        buy_tx = asset_map[asset][BUY].pop()
        print("Carryover:", buy_tx['IRS ID'])
        buy_volume += buy_tx['Total Amount']
        
    
    print("Capital Gain:", capital_gain)
    print("Buy Volume:", buy_volume)
    print("Sell Volume:", sell_volume)
    print("\n")
            
            
        

BTC
--------------------------------------------
-6.661338147750939e-16
-4.0000037504484e-08
Capital Gain: 12557.889626055592
Buy Volume: 10826342.760000005
Sell Volume: 10838900.649626052


ETH
--------------------------------------------
-1.27675647831893e-15
Carryover: Gemi 976
Capital Gain: 1854.1699999999548
Buy Volume: 987332.0300000007
Sell Volume: 989186.2000000003




# 4. Create Summary Report